The semi-analytical formula for a Heston call involves a complex integral :


$$\begin{aligned}  C=\frac{1}{2}\left(X_0-K e^{-r T}\right)+\frac{1}{\pi} \Re \int_0^{\infty}\left(e^{r T} \frac{\phi(u-\mathbf{i})}{\mathbf{i} u K^{\mathbf{i} u}}-K \frac{\phi(u)}{\mathbf{i} u K^{\mathbf{i} u}}\right) \mathrm{d} u \end{aligned}$$
With: 
* $\phi(u)=e^{r T} X_0^{\mathbf{i} u}\left(\frac{1-g e^{-d T}}{1-g}\right)^{-2 \frac{\theta \kappa}{\lambda^2}} \exp \left(\frac{\theta \kappa T}{\lambda^2}(\kappa-\rho \lambda \mathbf{i} u-d)+\frac{V_0}{\lambda^2}(\kappa-\rho \lambda \mathbf{i} u+d) \frac{1-e^{d T}}{1-g e^{d T}}\right), $
* $d=\sqrt{(\rho \lambda u \mathbf{i}-\kappa)^2+\lambda^2\left(\mathbf{i} u+u^2\right)}, $
* $g=\frac{\kappa-\rho \lambda \mathbf{i} u-d}{\kappa-\rho \lambda \mathbf{i} u+d} .$

The complex integral requires a careful discretization.

In [2]:
import numpy as np
from scipy.optimize import broyden1

In [14]:
# Initiale values
X0 = 95
V0 = 0.1
r = 0.03
kappa = 1.5768
theta=0.0398
lambd=0.575
rho=-0.5711

In [47]:
def HestonModel(kappa, theta, lambd, T, K, X0, V0, rho, r):
    I = complex(0, 1) # define a imaginary unit
    integral, umax, N = 0, 1000, 10000 # initialize the call price C and set numerical integration limits
    du = umax/N # define the stepsize for numerical integration
    
    # Constants for characteristic function
    aa = theta * kappa * T / lambd**2
    bb = -2 * theta * kappa / lambd**2
    
    for i in range(1, N):
        u2 = i * du # define itegration variable u2
        u1 = complex(u2, -1) # define shifted integration variable u1
        
        # Compute a1 and a2 based on the model parameters
        a1, a2 = rho * lambd * u1 * I, rho * lambd * u2 * I
        
        # Compute characteristic function terms
        d1 = np.sqrt((a1 - kappa)**2 + lambd**2 * (u1 * I + u1**2))
        d2 = np.sqrt((a2 - kappa)**2 + lambd**2 * (u2 * I + u2**2))
        
        # Compute g functions
        
        ## Avoid division by zero and numerical overflows
        dg1, dg2 = kappa - a1 + d1, kappa - a2 + d2
        
        g1 = np.where(dg1!=0, (kappa - a1 - d1) / dg1, 0) ## If denominator is zero, set g1 to 0
        g2 = np.where(dg2!=0, (kappa - a2 - d2) / dg2, 0)
        
        # Compute the terms of the characteristic functions
        dt1 = np.exp(-d1 * T)
        denom1 = np.where((1 - g1)!=0, (1 - g1 * dt1) / (1 - g1), 1)
        term11 = np.exp(r * T + (u1 * I) * np.log(X0) + bb * np.log(denom1))
        dt1 = np.exp(-d1 * T)
        frac1 = np.where((1 - g1 * dt1)!=0, (1 - dt1) / (1 - g1 * dt1), 1)
        term21 = np.exp(aa * (kappa - a1 - d1) + (V0 / lambd**2) * (kappa -a1 + d1) * frac1)
        
        dt2 = np.exp(-d2 * T)
        denom2 = np.where((1 - g2)!=0, (1 - g2 * dt2) / (1 - g2), 1)
        term21 = np.exp(r * T + (u2 * I) * np.log(X0) + bb * np.log(denom2))
        dt2 = np.exp(-d2 * T)
        frac2 = np.where((1 - g2 * dt2)!=0, (1 - dt2) / (1 - g2 * dt2), 1)
        term22 = np.exp(aa * (kappa - a2 - d2) + (V0 / lambd**2) * (kappa -a2 + d2) * frac2)
        
        # Compute characteristic functions ph1 and phi2
        phi1 = term11 * term21
        phi2 = term21 * term22
        
        # Compute the integral
        d = u2*I * K**(u2*I)
        integral += (np.exp(r*T) * phi1 - K * phi2) * du / d
    
    # Compute the final value
    C = (X0 - K * np.exp(-r * T)) / 2 + np.real(integral) / np.pi
    
    return C     

In [48]:
# Example of usage of heston()
T,K=2,100
call = HestonModel(kappa, theta, lambd, T, K, X0, V0, rho, r)
print("call = ",call, " put = ", call-X0+K*np.exp(-r*T))

call =  -80.47901959588134  put =  -81.30256623745647


In [31]:
import numpy as np

def HestonModel0(kappa, theta, lambd, T, K, X0, V0, rho, r):
    I = complex(0, 1)  # Define imaginary unit
    integral, umax, N = 0, 1000, 10000  # Initialize integral and set numerical integration limits
    du = umax / N  # Step size for numerical integration
    
    # Constants for characteristic function
    aa = theta * kappa * T / lambd**2
    bb = -2 * theta * kappa / lambd**2
    
    for i in range(1, N):
        u2 = i * du  # Define integration variable u2
        u1 = complex(u2, -1)  # Define shifted integration variable u1
        
        # Compute a1 and a2 based on model parameters
        a1, a2 = rho * lambd * u1 * I, rho * lambd * u2 * I
        
        # Compute characteristic function terms
        d1 = np.sqrt((a1 - kappa)**2 + lambd**2 * (u1 * I + u1**2))
        d2 = np.sqrt((a2 - kappa)**2 + lambd**2 * (u2 * I + u2**2))
        
        # Avoid division by zero and numerical overflows
        g1_denom = kappa - a1 + d1
        g2_denom = kappa - a2 + d2
        
        g1 = np.where(g1_denom != 0, (kappa - a1 - d1) / g1_denom, 0)  # If denominator is zero, set g1 to 0
        g2 = np.where(g2_denom != 0, (kappa - a2 - d2) / g2_denom, 0)
        
        # Compute exponential terms
        dt1 = np.exp(-d1 * T)
        dt2 = np.exp(-d2 * T)
        
        denom1 = np.where((1 - g1) != 0, (1 - g1 * dt1) / (1 - g1), 1)
        term11 = np.exp(r * T + (u1 * I) * np.log(X0) + bb * np.log(denom1))
        term21 = np.exp(aa * (kappa - a1 - d1) + (V0 / lambd**2) * (kappa - a1 + d1) * (1 - dt1) / (1 - g1 * dt1))
        
        denom2 = np.where((1 - g2) != 0, (1 - g2 * dt2) / (1 - g2), 1)
        term12 = np.exp(r * T + (u2 * I) * np.log(X0) + bb * np.log(denom2))
        term22 = np.exp(aa * (kappa - a2 - d2) + (V0 / lambd**2) * (kappa - a2 + d2) * (1 - dt2) / (1 - g2 * dt2))
        
        # Compute characteristic functions phi1 and phi2
        phi1 = term11 * term21
        phi2 = term12 * term22
        
        # Compute the integral
        d = u2 * I * K**(u2 * I)
        integral += (np.exp(r * T) * phi1 - K * phi2) * du / d
    
    # Compute the final value
    C = (X0 - K * np.exp(-r * T)) / 2 + np.real(integral) / np.pi
    
    return C


In [33]:
# Example of usage of heston()
T,K=2,100
call = HestonModel0(kappa, theta, lambd, T, K, X0, V0, rho, r)
print("call = ",call, " put = ", call-X0+K*np.exp(-r*T))

call =  28.63557761398665  put =  27.81203097241152


In [49]:
# example of calibration 
price1 = HestonModel0(kappa, theta, lambd, T, 90, X0, V0, rho, r)
price2 = HestonModel0(kappa, theta, lambd, T, 105, X0, V0, rho, r)
price3 = HestonModel0(kappa, theta, lambd, T, 110, X0, V0, rho, r)

def F(x):
    return [(price1 - HestonModel0(x[0],x[1],x[2],T,90, X0, V0, rho, r)), \
            (price2 - HestonModel0(x[0],x[1],x[2],T,105, X0, V0, rho, r)), \
            (price3 - HestonModel0(x[0],x[1],x[2],T,110, X0, V0, rho, r))]
x = broyden1(F, [1.4,0.03,0.5], f_tol=1e-14)
print("[kappa,theta,lambda] =",x)

C:\Users\asus\AppData\Local\Temp\ipykernel_14772\1623619327.py:48: RuntimeWarning: overflow encountered in scalar multiply
  integral += (np.exp(r * T) * phi1 - K * phi2) * du / d
C:\Users\asus\AppData\Local\Temp\ipykernel_14772\1623619327.py:48: RuntimeWarning: invalid value encountered in scalar multiply
  integral += (np.exp(r * T) * phi1 - K * phi2) * du / d
C:\Users\asus\AppData\Local\Temp\ipykernel_14772\1623619327.py:48: RuntimeWarning: overflow encountered in scalar subtract
  integral += (np.exp(r * T) * phi1 - K * phi2) * du / d
C:\Users\asus\AppData\Local\Temp\ipykernel_14772\1623619327.py:43: RuntimeWarning: overflow encountered in scalar multiply
  phi1 = term11 * term21
C:\Users\asus\AppData\Local\Temp\ipykernel_14772\1623619327.py:48: RuntimeWarning: invalid value encountered in scalar subtract
  integral += (np.exp(r * T) * phi1 - K * phi2) * du / d
C:\Users\asus\AppData\Local\Temp\ipykernel_14772\1623619327.py:43: RuntimeWarning: invalid value encountered in scalar mul

KeyboardInterrupt: 

In [17]:
def heston(kappa, theta, lambd, T, K, X0, V0, rho, r):
    I = complex(0, 1)  # Define imaginary unit
    P, umax, N = 0, 1000, 10000  # Initialize P and set numerical integration limits
    du = umax / N  # Step size for numerical integration
    
    # Constants for characteristic function
    aa = theta * kappa * T / lambd**2
    bb = -2 * theta * kappa / lambd**2
    
    for i in range(1, N):
        u2 = i * du  # Define integration variable u2
        u1 = complex(u2, -1)  # Define shifted integration variable u1
        
        # Compute a1 and a2 based on model parameters
        a1 = rho * lambd * u1 * I
        a2 = rho * lambd * u2 * I
        
        # Compute characteristic function terms
        d1 = np.sqrt((a1 - kappa)**2 + lambd**2 * (u1 * I + u1**2))
        d2 = np.sqrt((a2 - kappa)**2 + lambd**2 * (u2 * I + u2**2))
        
        # Avoid division by zero and numerical overflows
        g1_denom = kappa - a1 + d1
        g2_denom = kappa - a2 + d2
        g1 = np.where(g1_denom != 0, (kappa - a1 - d1) / g1_denom, 0)
        g2 = np.where(g2_denom != 0, (kappa - a2 - d2) / g2_denom, 0)
        
        # Compute exponential terms
        b1_exp = np.exp(-d1 * T)
        b2_exp = np.exp(-d2 * T)
        b1_denom = np.where(1 - g1 != 0, (1 - g1 * b1_exp) / (1 - g1), 1)
        b2_denom = np.where(1 - g2 != 0, (1 - g2 * b2_exp) / (1 - g2), 1)
        b1 = np.exp(u1 * I * (np.log(X0 / K) + r * T)) * b1_denom**bb
        b2 = np.exp(u2 * I * (np.log(X0 / K) + r * T)) * b2_denom**bb
        
        # Compute characteristic functions phi1 and phi2, avoiding overflow
        exp_term1 = np.exp(-d1 * T)
        exp_term2 = np.exp(-d2 * T)
        denom1 = np.where(1 - g1 * exp_term1 != 0, (1 - g1 * exp_term1), 1)
        denom2 = np.where(1 - g2 * exp_term2 != 0, (1 - g2 * exp_term2), 1)
        
        term1 = V0 * (kappa - a1 - d1) * (1 - exp_term1) / (denom1 * lambd**2)
        term2 = V0 * (kappa - a2 - d2) * (1 - exp_term2) / (denom2 * lambd**2)
        
        phi1 = b1 * np.exp(aa * (kappa - a1 - d1) + term1)
        phi2 = b2 * np.exp(aa * (kappa - a2 - d2) + term2)
        
        # Compute integral sum
        P += ((phi1 - phi2) / (u2 * I)) * du
    
    # Compute final option price
    return K * np.real((X0 / K - np.exp(-r * T)) / 2 + P / np.pi)

# Example usage (Ensure variables like X0, V0, rho, and r are defined before calling)


In [50]:
# Example of usage of heston()
T,K=2,100
call = heston(kappa, theta, lambd, T, K, X0, V0, rho, r)
print("call = ",call, " put = ", call-X0+K*np.exp(-r*T))

call =  12.356330803154563  put =  11.532784161579428


In [51]:
def F(x):
    return [(price1 - heston(x[0],x[1],x[2],T,90, X0, V0, rho, r)), \
            (price2 - heston(x[0],x[1],x[2],T,105, X0, V0, rho, r)), \
            (price3 - heston(x[0],x[1],x[2],T,110, X0, V0, rho, r))]
x = broyden1(F, [1.4,0.03,0.5], f_tol=1e-14)
print("[kappa,theta,lambda] =",x)

OverflowError: (34, 'Result too large')